In [56]:
import pandas as pd
from io import StringIO
import numpy as np

In [57]:
with open('../data/raw/2023_premier_league_match_data_raw.jsonl', 'r') as file:
    jsonl_string = file.read()
json_io = StringIO(jsonl_string)
data = pd.read_json(json_io, lines=True)
data.head()

,area,competition,season,id,utcDate,status,matchday,stage,group,lastUpdated,homeTeam,awayTeam,score,odds,referees
0,"{'id': 2072, 'name': 'England', 'code': 'ENG',...","{'id': 2021, 'name': 'Premier League', 'code':...","{'id': 1564, 'startDate': '2023-08-11', 'endDa...",435943,2023-08-11T19:00:00Z,FINISHED,1,REGULAR_SEASON,NaN,2023-12-12T15:20:40Z,"{'id': 328, 'name': 'Burnley FC', 'shortName':...","{'id': 65, 'name': 'Manchester City FC', 'shor...","{'winner': 'AWAY_TEAM', 'duration': 'REGULAR',...",{'msg': 'Activate Odds-Package in User-Panel t...,"[{'id': 11585, 'name': 'Craig Pawson', 'type':..."
1,"{'id': 2072, 'name': 'England', 'code': 'ENG',...","{'id': 2021, 'name': 'Premier League', 'code':...","{'id': 1564, 'startDate': '2023-08-11', 'endDa...",435944,2023-08-12T12:00:00Z,FINISHED,1,REGULAR_SEASON,NaN,2023-09-19T20:20:30Z,"{'id': 57, 'name': 'Arsenal FC', 'shortName': ...","{'id': 351, 'name': 'Nottingham Forest FC', 's...","{'winner': 'HOME_TEAM', 'duration': 'REGULAR',...",{'msg': 'Activate Odds-Package in User-Panel t...,"[{'id': 11605, 'name': 'Michael Oliver', 'type..."
2,"{'id': 2072, 'name': 'England', 'code': 'ENG',...","{'id': 2021, 'name': 'Premier League', 'code':...","{'id': 1564, 'startDate': '2023-08-11', 'endDa...",435945,2023-08-12T14:00:00Z,FINISHED,1,REGULAR_SEASON,NaN,2023-09-19T20:20:30Z,"{'id': 1044, 'name': 'AFC Bournemouth', 'short...","{'id': 563, 'name': 'West Ham United FC', 'sho...","{'winner': 'DRAW', 'duration': 'REGULAR', 'ful...",{'msg': 'Activate Odds-Package in User-Panel t...,"[{'id': 11309, 'name': 'Peter Bankes', 'type':..."
3,"{'id': 2072, 'name': 'England', 'code': 'ENG',...","{'id': 2021, 'name': 'Premier League', 'code':...","{'id': 1564, 'startDate': '2023-08-11', 'endDa...",435946,2023-08-12T14:00:00Z,FINISHED,1,REGULAR_SEASON,NaN,2023-09-19T20:20:30Z,"{'id': 397, 'name': 'Brighton & Hove Albion FC...","{'id': 389, 'name': 'Luton Town FC', 'shortNam...","{'winner': 'HOME_TEAM', 'duration': 'REGULAR',...",{'msg': 'Activate Odds-Package in User-Panel t...,"[{'id': 11556, 'name': 'David Coote', 'type': ..."
4,"{'id': 2072, 'name': 'England', 'code': 'ENG',...","{'id': 2021, 'name': 'Premier League', 'code':...","{'id': 1564, 'startDate': '2023-08-11', 'endDa...",435947,2023-08-12T14:00:00Z,FINISHED,1,REGULAR_SEASON,NaN,2023-09-19T20:20:30Z,"{'id': 62, 'name': 'Everton FC', 'shortName': ...","{'id': 63, 'name': 'Fulham FC', 'shortName': '...","{'winner': 'AWAY_TEAM', 'duration': 'REGULAR',...",{'msg': 'Activate Odds-Package in User-Panel t...,"[{'id': 11494, 'name': 'Stuart Attwell', 'type..."


In [58]:
matches = pd.DataFrame({
    'id': data['id'],
    'home_team_id': data['homeTeam'].apply(lambda x: x.get('id')),
    'home_team_name': data['homeTeam'].apply(lambda x: x.get('name')),
    'away_team_id': data['awayTeam'].apply(lambda x: x.get('id')),
    'away_team_name': data['awayTeam'].apply(lambda x: x.get('name')),
    'winner': data['score'].apply(lambda x: x.get('winner')),
    'home_goals': data['score'].apply(lambda x: x.get('fullTime').get('home')),
    'away_goals': data['score'].apply(lambda x: x.get('fullTime').get('away'))
})

matches.head()

,id,home_team_id,home_team_name,away_team_id,away_team_name,winner,home_goals,away_goals
0,435943,328,Burnley FC,65,Manchester City FC,AWAY_TEAM,0.0,3.0
1,435944,57,Arsenal FC,351,Nottingham Forest FC,HOME_TEAM,2.0,1.0
2,435945,1044,AFC Bournemouth,563,West Ham United FC,DRAW,1.0,1.0
3,435946,397,Brighton & Hove Albion FC,389,Luton Town FC,HOME_TEAM,4.0,1.0
4,435947,62,Everton FC,63,Fulham FC,AWAY_TEAM,0.0,1.0


In [59]:
home_team_id_list = matches['homeTeamID'].unique()
away_team_id_list = matches['awayTeamID'].unique()
full_team_id_list = np.unique(np.append(home_team_id_list, away_team_id_list))

teams = pd.DataFrame(full_team_id_list, columns=['id'])


# def goal_sum(row):
#     if row['id'] in matches['homeTeamID'].values():
#         return 'homeGoals'
#     elif row['id'] in matches['awayTeamID'].values():
#         return 'awayGoals'
#     else:
#         return None


# teams['goals_scored_fields'] = matches.apply(goal_sum, axis=1)
# teams






KeyError: 'homeTeamID'

In [ ]:
team_stats = {}

# Iterate through matches
for index, match in matches.iterrows():
    home_team_id, home_team_name = match['home_team_id'], match['home_team_name']
    away_team_id, away_team_name = match['away_team_id'], match['away_team_name']
    winner, home_goals, away_goals = match['winner'], match['home_goals'], match['away_goals']

    # Update home team statistics
    team_stats.setdefault(home_team_id, {'team_id': home_team_id, 'team_name': home_team_name, 'goals_scored': 0, 'goals_against': 0, 'total_points': 0})
    team_stats[home_team_id]['goals_scored'] += home_goals
    team_stats[home_team_id]['goals_against'] += away_goals
    team_stats[home_team_id]['total_points'] += 3 if winner == 'HOME' else 1 if winner == 'DRAW' else 0

    # Update away team statistics
    team_stats.setdefault(away_team_id, {'team_id': away_team_id, 'team_name': away_team_name, 'goals_scored': 0, 'goals_against': 0, 'total_points': 0})
    team_stats[away_team_id]['goals_scored'] += away_goals
    team_stats[away_team_id]['goals_against'] += home_goals
    team_stats[away_team_id]['total_points'] += 3 if winner == 'AWAY' else 1 if winner == 'DRAW' else 0

# Create the final standings DataFrame
standings_df = pd.DataFrame(list(team_stats.values()))

# Display the standings DataFrame
print(standings_df)

    team_id                   team_name  goals_scored  goals_against  \
0        63                   Fulham FC            27             53   
1        57                  Arsenal FC            55             39   
2       354           Crystal Palace FC            41             66   
3       340              Southampton FC            47             68   
4        64                Liverpool FC            68             42   
5       341             Leeds United FC            62             54   
6       563          West Ham United FC            62             47   
7        67         Newcastle United FC            46             62   
8        74     West Bromwich Albion FC            35             76   
9       338           Leicester City FC            68             50   
10       73        Tottenham Hotspur FC            68             45   
11       62                  Everton FC            47             48   
12      356         Sheffield United FC            20           

In [61]:
team_stats = pd.DataFrame(columns=['team_id', 'team_name', 'goals_scored', 'goals_against', 'total_points'])

# Update team statistics based on home teams
home_team_stats = matches.groupby('home_team_id').agg({
    'home_team_name': 'first',
    'home_goals': 'sum',
    'away_goals': 'sum',
    'winner': lambda x: x.eq('Home').sum() * 3 + x.eq('Draw').sum()
}).reset_index()

# Update team statistics based on away teams
away_team_stats = matches.groupby('away_team_id').agg({
    'away_team_name': 'first',
    'away_goals': 'sum',
    'home_goals': 'sum',
    'winner': lambda x: x.eq('Away').sum() * 3 + x.eq('Draw').sum()
}).reset_index()

# Combine home and away team statistics
team_stats = home_team_stats.rename(columns={'home_team_id': 'team_id', 'home_team_name': 'team_name'}).append(
    away_team_stats.rename(columns={'away_team_id': 'team_id', 'away_team_name': 'team_name'}),
    ignore_index=True
)

# Calculate total points
team_stats['total_points'] = team_stats['winner']

# Display the final standings DataFrame
print(team_stats[['team_id', 'team_name', 'goals_scored', 'goals_against', 'total_points']])

AttributeError: 'DataFrame' object has no attribute 'append'